In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import pandas as pd
import numpy as np

2024-05-16 15:32:54.402359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
## load data
x_train = pd.read_csv("../../../data/04/processed_for_ml/binary_class/x_train_taxahfe_no_sf.csv", index_col=0, header=0)
y_train = pd.read_csv("../../../data/04/processed_for_ml/binary_class/y_train_class.csv", index_col = 0, header = 0 )
sw_train = pd.read_csv("../../../data/04/processed_for_ml/binary_class/sw_train_taxahfe.csv", index_col = 0, header = 0 )
x_test = pd.read_csv("../../../data/04/processed_for_ml/binary_class/x_test_no_sf.csv", index_col=0, header=0)
y_test = pd.read_csv("../../../data/04/processed_for_ml/binary_class/y_test_class.csv", index_col = 0, header = 0 )
x_test = x_test[x_train.columns] # reordered columns to match (column ordered changed during taxahfe)

x_train = x_train.sort_index(axis = 0)
y_train = y_train.sort_index(axis = 0)
sw_train = sw_train.sort_index(axis = 0)
x_test = x_test.sort_index(axis = 0)
y_test = y_test.sort_index(axis = 0)

x_train = x_train.reset_index()
y_train = y_train.reset_index()
sw_train = sw_train.reset_index()
x_test = x_test.reset_index()
y_test = y_test.reset_index()

x_train = x_train.drop(columns='SEQN')
y_train = y_train.drop(columns='SEQN')
sw_train = sw_train.drop(columns='SEQN')
x_test = x_test.drop(columns='SEQN')
y_test = y_test.drop(columns='SEQN')

x_train = np.array(x_train)
y_train = np.array(y_train)
sample_weights = np.array(sw_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [4]:
keras.utils.set_random_seed(0)

In [5]:
acc = keras.metrics.Accuracy()

In [6]:
# baseline model
def model_builder(hp):
	# create model
    model = keras.Sequential()
    model.add(keras.layers.BatchNormalization(input_shape=(37,)))

	# Tune the number of units in the first Dense layer
    hp_units_1 = hp.Int('units_1', min_value=32, max_value=256, step=16)
    model.add(keras.layers.Dense(units=hp_units_1, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

	# Compile model
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), sample_weight_mode=sample_weights, metrics=['accuracy'])
    return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='tune',
                     project_name='binary_class')

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
lr_curve = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=5)

In [9]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early, lr_curve])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_1')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 08s]
val_accuracy: 0.7545982599258423

Best val_accuracy So Far: 0.7676669955253601
Total elapsed time: 00h 01m 57s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.01.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

acc_per_epoch = history.history['val_accuracy']
best_epoch = acc_per_epoch.index(max(acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5614 - accuracy: 0.7184 - val_loss: 0.5317 - val_accuracy: 0.7435
Epoch 2/50
259/259 [==============================] - 0s 2ms/step - loss: 0.5299 - accuracy: 0.7337 - val_loss: 0.5044 - val_accuracy: 0.7551
Epoch 3/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5247 - accuracy: 0.7367 - val_loss: 0.5017 - val_accuracy: 0.7507
Epoch 4/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5215 - accuracy: 0.7435 - val_loss: 0.4814 - val_accuracy: 0.7667
Epoch 5/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5154 - accuracy: 0.7463 - val_loss: 0.4854 - val_accuracy: 0.7652
Epoch 6/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5166 - accuracy: 0.7408 - val_loss: 0.4836 - val_accuracy: 0.7585
Epoch 7/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5132 - accuracy: 0.7511 - val_loss: 0.4868 - val_accuracy: 0.7594
Epoch 

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/4
259/259 [==============================] - 1s 2ms/step - loss: 0.5649 - accuracy: 0.7189 - val_loss: 0.5324 - val_accuracy: 0.7454
Epoch 2/4
259/259 [==============================] - 0s 1ms/step - loss: 0.5304 - accuracy: 0.7332 - val_loss: 0.5018 - val_accuracy: 0.7546
Epoch 3/4
259/259 [==============================] - 0s 2ms/step - loss: 0.5252 - accuracy: 0.7373 - val_loss: 0.4997 - val_accuracy: 0.7531
Epoch 4/4
259/259 [==============================] - 0s 2ms/step - loss: 0.5227 - accuracy: 0.7416 - val_loss: 0.4821 - val_accuracy: 0.7652


In [12]:
b_acc = keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)

In [13]:
pred = hypermodel(x_test)
b_acc.update_state(y_test, pred)
result = b_acc.result()
result.numpy()

0.7401239

In [14]:
# save model
hypermodel.save('model/binary_class.keras')